# UCF Communities Quarterly Reporting

## This goal of this script is to calculate the total acres of communities assisted by Urban & Community Forestry (UCF) each quarter. This is an additional step to the FRD Accomplishments Map due each quarter. You can find more detailed information on the README in Github https://github.com/jgorman-tfs/FRD-Accomplishments <br>
### Before you begin, please ensure you have done the following steps
1. Received the quarterly division reporting from Mac
2. Organized the communities into ONE SHEET, ensuring no duplicates and no categories that should be excluded (e.g. phone calls). Read more about data clean up in the README
3. Have only 2 columns: ideally one as CityName and one as AssistType.
4. You MUST have a U.S. Census Places layer with a column for the County that Place is in. 

### You should only need to alter the following cell. Ensure the paths are correct and the correct sheet name is used.

In [ ]:
import pandas as pd
import os
#Quarter and fiscal year
qtr = "Q6_2024"
#Set folder path to new quarter folder
folder_path = r'D:\ArcGIS_Projects\FRDAccomplishments\Testing'
#Set the geodatabase path
gdb = r'D:\ArcGIS_Projects\FRDAccomplishments\FRDAccomplishments.gdb'
#This shapefile is provided in sharepoint. If your using your own, you MUST calculate a new field with the County names of each city.
cities = "Texas_Places_WithCounties"
#Path to the excel file provided by Mac
excel_path = r"D:\ArcGIS_Projects\FRDAccomplishments\Testing\Q4 Division Reporting.xlsx"
#Sheet name of said excel file that has the edited data
sheet_name = "AllCommunities2"
#This is a temperory csv, it doesn't really matter where it goes
output_path = os.path.join(r'D:\ArcGIS_Projects\FRDAccomplishments', 'Q6FY24Cities.csv')

In [ ]:
'''
This block will convert the excel sheet into a pandas dataframe and rename the columns for consistency,
removes spaces, and converts it back to excel to be converted to an arcgis pro table
'''
aprx = arcpy.mp.ArcGISProject("CURRENT")
aprx.save()

# create a dataframe from the excel file and sheet
df = pd.read_excel(excel_path, sheet_name=sheet_name)

# Rename columns
old_column_list = df.columns.tolist()
print("old column headers:", old_column_list)
new_column_names = {
    old_column_list[0]: "CityName",
    old_column_list[1]: "AssistType"
}
df.rename(columns=new_column_names, inplace=True)

# Remove spaces from the 'CityName' column
df['CityName'] = df['CityName'].str.replace(' ', '', regex=False)
print(df.head(5))

# Convert the DataFrame to csv  file and print its location
df.to_csv(output_path, index=False)
print(f"Excel file saved to: {output_path}")

#create a table in ArcGIS Pro from the csv
arcpy.TableToTable_conversion(output_path, gdb, f"{qtr}_Cities")

# CRITICAL STEP
### Open the newly created ArcGIS Pro table and look for null values in the testing column. Figure out why they are not matching to the Texas_Places_WithCounties. It is likely they are either misspelled or are actually counties and not cities. If they are counties, you can delete them.

### Once you have fixed the issue, delete the testing column and re-run this cell to check. Repeat until there are no null values in the testing column. 

In [ ]:
#Join field from Texas_Places_WithCounties to match the city name to the county it resides in
in_data = f"{gdb}\\{qtr}_Cities"
in_field = "CityName"
join_table = f"{gdb}\\{cities}"
join_field = "CityName"
fields = "Testing"
arcpy.management.JoinField(in_data, in_field, join_table, join_field, fields)

In [ ]:
in_table = f"{gdb}\\{qtr}_Cities"
field_name = f"{qtr}_Acres"
field_type = "FLOAT"
field_precision = 2
arcpy.management.AddField(in_table, field_name, field_type, field_precision)

expression = 0
arcpy.management.CalculateField(in_table, field_name, expression)

In [ ]:
'''This block performs a field join back to the Texas_Places_WithCounties so that the acres can be calculated'''

texas_cities = f"{gdb}\\{cities}"
in_field = "CityName"
imported_table = f"{gdb}\\{qtr}_Cities"
join_field = "CityName"
fields = field_name
arcpy.management.JoinField(texas_cities, in_field, imported_table, join_field, fields)

In [ ]:
'''This block will do a final check to make sure Texas_Places_WithCounties 
has the correct amount of cities to calculate. If it doesn't, you will need find out why they are not matching '''

arcpy.management.SelectLayerByAttribute(imported_table, "NEW_SELECTION", f"{fields} = 0")
num_cities_intable = int(arcpy.management.GetCount(imported_table)[0])
print(f"Number of cities in the Imported Table: {num_cities_intable}")
arcpy.management.SelectLayerByAttribute(imported_table, "CLEAR_SELECTION")

arcpy.management.SelectLayerByAttribute(cities, "NEW_SELECTION", f"{field_name} = 0")
num_cities_incities = int(arcpy.management.GetCount(cities)[0])
print(f"Number of cities in the Texas_Places_WithCounties: {num_cities_incities}")
arcpy.management.SelectLayerByAttribute(cities, "CLEAR_SELECTION")

if num_cities_intable != num_cities_incities:
    print("There is something wrong. The number of cities in the imported table and the feature layer do not match.")
else:
    print("Continue onto the next step")

In [ ]:
'''This block will select the appropriate citiies and calculate their acres'''

arcpy.management.SelectLayerByAttribute(cities, "NEW_SELECTION", f"{field_name} = 0")
num_cities_incities = int(arcpy.management.GetCount(cities)[0])
print(f"Number of cities in the Texas_Places_WithCounties: {num_cities_incities}")

length_unit = "MILES_US"
area_unit = "ACRES_US"

# Define the WKID (Well-Known ID) of the coordinate system you want to use
wkid = 3665  # Example: WGS 1984 Geographic Coordinate System

try:
    # Create a spatial reference object using the specified WKID
    spatial_ref = arcpy.SpatialReference(3665)

    # Calculate geometry attributes with specified coordinate system and units
    arcpy.management.CalculateGeometryAttributes(cities,
                                                 [[field_name, "AREA_GEODESIC"]],
                                                 length_unit, area_unit,
                                                 spatial_ref)

    print("Geometry attributes calculated successfully.")

except arcpy.ExecuteError as e:
    print(f"ExecuteError: {e}")
except Exception as e:
    print(f"Error occurred: {e}")

In [ ]:
'''This block will sum the the acres into a new table'''

community_acres = f"{gdb}\\{qtr}_CommunityAcres_Total"
arcpy.analysis.Statistics(cities, community_acres, [[field_name, "SUM"]])
arcpy.management.SelectLayerByAttribute(cities, "CLEAR_SELECTION")